## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-18-21-50-35 +0000,nyt,What Jimmy Kimmel’s Suspension Says About the ...,https://www.nytimes.com/2025/09/18/business/me...
1,2025-09-18-21-47-02 +0000,bbc,Why students like me are happier living at home,https://www.bbc.com/news/articles/c15kwylxjglo...
2,2025-09-18-21-44-59 +0000,nypost,"NYC student, 16, pledges to shoot up school on...",https://nypost.com/2025/09/18/us-news/nyc-stud...
3,2025-09-18-21-43-34 +0000,nypost,D4vd’s eerie unreleased ‘Celeste’ song surface...,https://nypost.com/2025/09/18/us-news/d4vds-un...
4,2025-09-18-21-42-27 +0000,nyt,Texas Man Dies in Fall From Alaska Mountain Trail,https://www.nytimes.com/2025/09/18/us/alaska-h...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2459/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,71
1,kimmel,28
0,jimmy,22
133,kirk,21
241,new,20


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
169,2025-09-18-15-10-04 +0000,nypost,Trump claims Jimmy Kimmel was taken off air fo...,https://nypost.com/2025/09/18/us-news/trump-cl...,196
73,2025-09-18-19-53-00 +0000,wsj,The Trump administration is putting the weight...,https://www.wsj.com/politics/policy/jimmy-kimm...,168
0,2025-09-18-21-50-35 +0000,nyt,What Jimmy Kimmel’s Suspension Says About the ...,https://www.nytimes.com/2025/09/18/business/me...,164
63,2025-09-18-20-13-00 +0000,wsj,The Trump administration asked the Supreme Cou...,https://www.wsj.com/politics/policy/trump-admi...,162
66,2025-09-18-20-10-29 +0000,latimes,Behind the decision to bench Jimmy Kimmel: Tru...,https://www.latimes.com/entertainment-arts/bus...,143


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
169,196,2025-09-18-15-10-04 +0000,nypost,Trump claims Jimmy Kimmel was taken off air fo...,https://nypost.com/2025/09/18/us-news/trump-cl...
63,91,2025-09-18-20-13-00 +0000,wsj,The Trump administration asked the Supreme Cou...,https://www.wsj.com/politics/policy/trump-admi...
120,65,2025-09-18-17-48-40 +0000,nypost,D’oh tech! Britain’s new tech secretary Liz K...,https://nypost.com/2025/09/18/us-news/britains...
304,56,2025-09-18-01-41-00 +0000,wsj,When the Federal Reserve cut interest rates We...,https://www.wsj.com/economy/central-banking/fe...
339,54,2025-09-17-23-03-00 +0000,wsj,Some military lawyers and other Defense Depart...,https://www.wsj.com/politics/national-security...
168,52,2025-09-18-15-15-02 +0000,nypost,Trump admits Putin ‘let me down’ on Ukraine pe...,https://nypost.com/2025/09/18/us-news/trump-ad...
88,51,2025-09-18-19-04-07 +0000,bbc,First migrant deported to France under 'one in...,https://www.bbc.com/news/articles/ckg653r06jgo...
3,45,2025-09-18-21-43-34 +0000,nypost,D4vd’s eerie unreleased ‘Celeste’ song surface...,https://nypost.com/2025/09/18/us-news/d4vds-un...
286,40,2025-09-18-03-38-00 +0000,wsj,President Trump is frustrated with Israeli Pri...,https://www.wsj.com/world/middle-east/trump-ne...
73,40,2025-09-18-19-53-00 +0000,wsj,The Trump administration is putting the weight...,https://www.wsj.com/politics/policy/jimmy-kimm...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
